## Balance Sheets - ETL Process

In [3]:
# Import Libraries
import time
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo

############################################################

# Delete collection before inserting new data
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client.Stocks_db.balance_sheets.drop()

############################################################

###############  CBA  ###############
##### Extract #####
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

stock = 'CBA'
url = f'https://au.finance.yahoo.com/quote/{stock}.AX/balance-sheet?p={stock}.AX'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
balance_sheet_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
balance_sheet_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in balance_sheet_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in balance_sheet_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
balance_table = pd.DataFrame(table_rows, columns=table_headers_list)
##### Extract #####

##### Transform #####
# Remove all the rows without values based on any of the financial year columns
balance_table_new = balance_table[balance_table['29/06/2021']!=''].copy()
# Replace all fields containing '-' with 0
balance_table_new = balance_table_new.replace('-',0)
# Set the index to Breakdown
balance_table_new = balance_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
balance_table_new_17to18 = balance_table_new[['29/06/2018']].copy()
#2018-2019
balance_table_new_18to19 = balance_table_new[['29/06/2019']].copy()
#2019-2020
balance_table_new_19to20 = balance_table_new[['29/06/2020']].copy()
#2020-2021
balance_table_new_20to21 = balance_table_new[['29/06/2021']].copy()

# Convert them into dictionaries
balance_table_new_17to18_dict = balance_table_new_17to18.to_dict()['29/06/2018']
balance_table_new_18to19_dict = balance_table_new_18to19.to_dict()['29/06/2019']
balance_table_new_19to20_dict = balance_table_new_19to20.to_dict()['29/06/2020']
balance_table_new_20to21_dict = balance_table_new_20to21.to_dict()['29/06/2021']
##### Transform #####

##### Load #####
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

stock_dict = {}

# Insert into MongoDB
stock_balance_reports = {'29/06/2018':balance_table_new_17to18_dict,
                         '29/06/2019':balance_table_new_18to19_dict,
                         '29/06/2020':balance_table_new_19to20_dict,
                         '29/06/2021':balance_table_new_20to21_dict}

stock_dict[stock] = stock_balance_reports

client.Stocks_db.balance_sheets.insert_one(stock_dict)
##### Load #####
###############  CBA  ###############

############################################################

###############  ANZ  ###############
##### Extract #####
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

stock = 'ANZ'
url = f'https://au.finance.yahoo.com/quote/{stock}.AX/balance-sheet?p={stock}.AX&.tsrc=fin-srch'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
balance_sheet_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
balance_sheet_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in balance_sheet_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in balance_sheet_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
balance_table = pd.DataFrame(table_rows, columns=table_headers_list)
##### Extract #####

##### Transform #####
# Remove all the rows without values based on any of the financial year columns
balance_table_new = balance_table[balance_table['29/09/2020']!=''].copy()
# Replace all fields containing '-' with 0
balance_table_new = balance_table_new.replace('-',0)
# Set the index to Breakdown
balance_table_new = balance_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
balance_table_new_17to18 = balance_table_new[['29/09/2018']].copy()
#2018-2019
balance_table_new_18to19 = balance_table_new[['29/09/2019']].copy()
#2019-2020
balance_table_new_19to20 = balance_table_new[['29/09/2020']].copy()

# Convert to dictionaries
balance_table_new_17to18_dict = balance_table_new_17to18.to_dict()['29/09/2018']
balance_table_new_18to19_dict = balance_table_new_18to19.to_dict()['29/09/2019']
balance_table_new_19to20_dict = balance_table_new_19to20.to_dict()['29/09/2020']
##### Transform #####

##### Load #####
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

stock_dict = {}

# Insert into MongoDB
stock_balance_reports = {'29/06/2018':balance_table_new_17to18_dict,
                         '29/06/2019':balance_table_new_18to19_dict,
                         '29/06/2020':balance_table_new_19to20_dict}

stock_dict[stock] = stock_balance_reports

client.Stocks_db.balance_sheets.insert_one(stock_dict)
##### Load #####
###############  ANZ  ###############

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\James\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\James\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
